In [3]:
"""
Important Stuff
"""

# OpenCV is a leading open source library for computer vision. To use it, we need to import it
# The easiest way to install OpenCV is by installing Anaconda distribution of python available for free here:
# https://www.anaconda.com/distribution/
import cv2

# Numpy is the leading python numerical library
import numpy as np

# Webcam index, this should be 0 for most laptops
DEVICE = 4

# We shrink outputput from the webcam in order to achieve a higher FPS
WIDTH = 640
HEIGHT = 480

"""
User Interface Stuff (You can ignore this)
"""
import IPython
import ipywidgets as widgets
from IPython.display import display

ipython = IPython.get_ipython()

style = {'description_width': 'initial'}

w_image = widgets.Image(width=WIDTH, height=HEIGHT, format='png',
                        layout=widgets.Layout(width='100%'))

w_rescale = widgets.Checkbox(
    value=False,
    description='Rescale Intensity',
    style=style
)
w_clip_std = widgets.FloatSlider(min=0.5, max=10, value=10.0, step=0.1, 
                                             description='Rescale: Clip MADs from Median',
                                             layout=widgets.Layout(width='100%'),
                                             style=style)
w_drop = widgets.Dropdown(options=[
    'No Equalization',
    'Histogram Equalization',
    'Adaptive Histogram Equalization',
], style=style)

w_blur = widgets.Checkbox(
    value=False,
    description='Gaussian Blur',
    style=style
)

vbox = widgets.VBox([w_image, w_rescale, w_clip_std, w_drop, w_blur])

display(vbox)

In [4]:
cap = cv2.VideoCapture(DEVICE)

try:
    while True:
        # Have to call this to get update values from sliders / dropdowns
        ipython.kernel.do_one_iteration()
        
        # Read the frame from the camera
        ret, frame = cap.read()
        
        # Resize the frame to a lower resolution for performance reasons
        img_original = cv2.resize(frame, (WIDTH, HEIGHT))

        # Convert to LAB color space so we can act on overall image intensity rather than each color seperately
        # https://en.wikipedia.org/wiki/CIELAB_color_space
        img_lab = cv2.cvtColor(img_original, cv2.COLOR_BGR2LAB)
        lab_planes = cv2.split(img_lab)
        
        img_intensity = lab_planes[0]
        
        if w_rescale.value:
                        
            # Convert from uint8 to floating point when scaling values
            img_intensity = img_intensity.astype(np.float32)
            
            # Clip statistical outliers
            
            med_val = np.median(img_intensity)
            mad_val = np.median(np.abs(img_intensity - med_val))
            std_val = np.std(img_intensity)
            
            clip_min = med_val - mad_val*w_clip_std.value
            clip_max = med_val + mad_val*w_clip_std.value
            
            img_intensity = np.clip(img_intensity, clip_min, clip_max)
            
            # Rescale between 0 and 1
            img_intensity = img_intensity - np.min(img_intensity)
            img_intensity = img_intensity / np.max(img_intensity)
            
            # Convert back to uint8
            img_intensity = img_intensity * 255
            img_intensity = img_intensity.astype(np.uint8)

        if w_drop.value == 'Adaptive Histogram Equalization':
            clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(4, 4))
            img_intensity = clahe.apply(img_intensity)
        elif w_drop.value == 'Histogram Equalization':
            img_intensity = cv2.equalizeHist(img_intensity)

        if w_blur.value:
            img_intensity = cv2.GaussianBlur(img_intensity, (5,5), 0)
            
        lab_planes[0] = img_intensity
            
        lab = cv2.merge(lab_planes)
        draw_image = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

        # Visualization Code
        result, img_png = cv2.imencode('.png', draw_image)
        w_image.value = img_png.tobytes()


        
except KeyboardInterrupt:
    pass
finally:
    cap.release()
